In [1]:
ENV["JULIA_REVISE"] = "manual" 
using Revise
using Probabilistic
using BenchmarkTools

In [2]:
revise()

In [3]:
brownian = @model N begin
    σ ~ Beta(2, 3)
    x = zeros(N)
    for i = 2:N
       x[i] ~ Normal(x[i-1], σ^2)
    end
end

N = 20
X = cumsum((0.75^2)*randn(N))

@benchmark sample(brownian(N) |> condition(:x => X) |> observe(:σ), Metropolis(1_000))

BenchmarkTools.Trial: 
  memory estimate:  8.50 MiB
  allocs estimate:  247384
  --------------
  minimum time:     229.142 ms (0.00% GC)
  median time:      235.948 ms (0.00% GC)
  mean time:        236.313 ms (0.75% GC)
  maximum time:     265.467 ms (0.00% GC)
  --------------
  samples:          22
  evals/sample:     1

In [4]:
mix = @model N begin
    prob ~ Beta(2, 2)

    mean0 ~ Normal(-1,1)
    mean1 ~ Normal(1,1)

    x = zeros(N)
    for i = 1:N
        x[i] ~ BiMix(prob, Normal(mean0, 0.2), Normal(mean1, 0.2))
    end
end

obs = [-1.7, -1.8, -2.01, -2.4, 1.9, 1.8]
N = length(obs)

@benchmark sample(mix(N) |> condition(:x => obs) |> observe(:mean0, :mean1), Metropolis(10_000))

BenchmarkTools.Trial: 
  memory estimate:  40.46 MiB
  allocs estimate:  1129603
  --------------
  minimum time:     825.292 ms (0.60% GC)
  median time:      830.404 ms (1.19% GC)
  mean time:        852.235 ms (1.06% GC)
  maximum time:     923.151 ms (1.08% GC)
  --------------
  samples:          6
  evals/sample:     1